In [1]:
import pandas

In [2]:
prediction = pandas.read_csv("./ensemble_train_prediction.csv")

In [15]:
prediction.describe()

,Id,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents,SeriousDlqin2yrs,DebtAbsolute,MonthlyBalance,BalancePerPerson,Prediction,probability,0 probability,1 probability
count,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,3.000000e+04,30000.000000,30000.000000,30000.000000,30000.000000,29237.000000,30000.000000,30000.000000,3.000000e+04,2.923700e+04,30000.000000,30000.000000,30000.000000,30000.000000
mean,75184.962033,5.221595,52.171300,0.409767,349.158550,5.362946e+03,8.451967,0.264767,1.015267,0.236267,0.757157,0.068567,1978.177339,3.384769e+03,2.382365e+03,0.019767,0.936945,0.933041,0.066959
std,43286.522991,219.424090,14.795051,4.122478,2526.704877,1.347350e+04,5.157297,4.105008,1.129399,4.085576,1.111296,0.252720,3050.460551,1.349694e+04,1.260885e+04,0.139200,0.097992,0.114013,0.114013
min,1.000000,0.000000,21.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-3.264420e+05,-3.264420e+05,0.000000,0.500100,0.261400,0.001920
25%,37689.000000,0.029766,41.000000,0.000000,0.170918,1.600000e+03,5.000000,0.000000,0.000000,0.000000,0.000000,0.000000,521.000000,2.014810e+02,3.293302e+02,0.000000,0.932330,0.932330,0.008200
50%,74657.500000,0.155847,52.000000,0.000000,0.362473,4.381500e+03,8.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1524.813544,2.760143e+03,1.699424e+03,0.000000,0.978710,0.978710,0.021290
75%,112785.250000,0.565905,63.000000,0.000000,0.838320,7.416000e+03,11.000000,0.000000,2.000000,0.000000,1.000000,0.000000,2808.093979,5.181260e+03,3.532003e+03,0.000000,0.991800,0.991800,0.067670
max,150000.000000,29110.000000,103.000000,98.000000,326442.000000,1.794060e+06,57.000000,98.000000,26.000000,98.000000,10.000000,1.000000,326442.000000,1.789080e+06,1.789080e+06,1.000000,0.998080,0.998080,0.738600


We create the confusion matrix logic and the method to get the result.

In [3]:
def confusion_matrix(row):
    x = "TN"
    if row['SeriousDlqin2yrs'] == 1:
        if row['Prediction'] == 1:
            x = "TP"
        else:
            x = "FN"
    else:
        if row['Prediction'] == 1:
            x = "FP"
    return x

In [4]:
def get_confusion_matrix(): 
    prediction['Error'] = prediction.apply(confusion_matrix, axis=1)
    return prediction['Error'].value_counts()

In [5]:
print(get_confusion_matrix())

TN    27708
FN     1699
TP      358
FP      235
Name: Error, dtype: int64


To get the **Accuracy** we add the **True Positives** and **True Negatives** and we divide them by **sample size/100**.

In [6]:
final_confusion_matrix = get_confusion_matrix()
accuracy = (final_confusion_matrix['TP'] + final_confusion_matrix['TN'])/300
print(accuracy)

93.55333333333333


We can have a look at the **100 largest errors**, with the **highest probability**.

In [7]:
filtered = prediction.loc[prediction['Error'].isin(["FN", "FP"])]

filtered = (filtered.nlargest(100, 'probability'))

From the given cost matrix we evaluate the best threshold to maximize our cash gain.

In [8]:
def cost_threshold(threshold):
    prediction_threshold = prediction['1 probability'] > threshold

    P_pred = prediction_threshold
    N_pred = ~prediction_threshold

    P_real = (prediction['SeriousDlqin2yrs'] == 1)
    N_real = (prediction['SeriousDlqin2yrs'] == 0)

    TP = len(prediction.loc[P_pred & P_real])
    TN = len(prediction.loc[N_pred & N_real])
    FP = len(prediction.loc[P_pred & N_real])
    FN = len(prediction.loc[N_pred & P_real])

    TP_cost = 0
    FN_cost = -2500
    FP_cost = -500
    TN_cost = 500

    cost = TP * TP_cost
    cost += TN * TN_cost
    cost += FP * FP_cost
    cost += FN * FN_cost

    return cost

With the **matplotlib** library we draw a plot to get the maximum cash gain value with it's matching threshold.

In [9]:
import matplotlib.pyplot as plt
table = [cost_threshold(n/1000) for n in range(0,1000)]
plt.plot(table)

max_val = max(table)
max_threshold = table.index(max_val)/3000
print(f'Le gain maximum est {max_val} et il est obtenu avec un seuil de {max_threshold}')

Le gain maximum est 9970500 et il est obtenu avec un seuil de 0.09433333333333334


To get the **AUC** we compare the **TP** probability score to the **TN** probability score.

In [17]:
def auc_method():
    auc_ratio = 0
    auc_count = 0
    auc_p_filter = prediction.loc[prediction['Error'].isin(["TP", "FN"])]
    auc_n_filter = prediction.loc[prediction['Error'].isin(["TN", "FP"])]
    
    for p_value in auc_p_filter['1 probability']:
        for n_value in auc_n_filter['1 probability']:
            if p_value > n_value:
                auc_ratio += 1
            auc_count += 1

    print(f"Nombre de comparaisons : {auc_ratio}")
    return auc_ratio/auc_count

In [18]:
print(auc_method())

Nombre de comparaisons : 49267257
0.8571386145812389
